In [1]:
from os import getcwd
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input, Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
train_dir = getcwd() + '/images/train/'
test_dir = getcwd() + '/images/test/'
testsplit = .2

In [3]:
seed = 42
targetx = 299
targety = 299
batch_size = 32

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (299, 299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_generator = test_datagen.flow_from_directory(test_dir,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [12]:
base_model = Xception(weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = False
    
# add a global spatial average pooling layer
x = base_model.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

# and a logistic layer -- let's say we have NUM_CLASSES classes
predictions = Dense(120, activation='softmax')(x)

# this is the model we will train
model = keras.Model(inputs=base_model.input, outputs=predictions)

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_7[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [13]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
len(train_generator)

In [14]:
model.fit_generator(
  train_generator,
  validation_data=test_generator,
  epochs=25,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
 35/375 [=>............................] - ETA: 34:45 - loss: 3.0383 - accuracy: 0.3911

KeyboardInterrupt: 